<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2E_Config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2E - CONFIGURACION COMPARTIDA
================================================================================

Trabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion
                        en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universitat Oberta de Catalunya (UOC)
Master: Data Science
Fecha: Diciembre 2025

Descripcion:
    Configuracion centralizada para todas las visualizaciones de Fase 2E.
    Incluye estilos, paletas de colores, rutas y funciones auxiliares compartidas.

USO EN GOOGLE COLAB:

    # PASO 1: Montar Drive (ejecutar en celda separada)
    from google.colab import drive
    drive.mount('/content/drive')

    # PASO 2: Cambiar al directorio de trabajo
    %cd /content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones

    # PASO 3: Importar configuracion
    from analisis_fase_2e_config import *

    # PASO 4: Inicializar (verifica rutas y crea directorios)
    estado = inicializar_fase2e()

    # PASO 5: Ejecutar bloques
    from analisis_fase_2e_bloque1 import ejecutar_bloque1
    resultados = ejecutar_bloque1()

ESTRUCTURA DE ARCHIVOS EN DRIVE:

    /content/drive/MyDrive/TFM/
    ├── 3_Analisis/
    │   └── fase2e_visualizaciones/
    │       ├── 03_analisis_fase_2e_config.py      <- Este archivo
    │       ├── 03_analisis_fase_2e_bloque1.py
    │       ├── 03_analisis_fase_2e_bloque2.py
    │       ├── 03_analisis_fase_2e_bloque3.py
    │       └── 03_analisis_fase_2e_bloque4.py

================================================================================
"""

"\n================================================================================\nFASE 2E - CONFIGURACION COMPARTIDA\n================================================================================\n\nTrabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion\n                        en Fotografia de Retrato\n\nAutor: Jesus L.\nUniversidad: Universitat Oberta de Catalunya (UOC)\nMaster: Data Science\nFecha: Diciembre 2025\n\nDescripcion:\n    Configuracion centralizada para todas las visualizaciones de Fase 2E.\n    Incluye estilos, paletas de colores, rutas y funciones auxiliares compartidas.\n\nUSO EN GOOGLE COLAB:\n    \n    # PASO 1: Montar Drive (ejecutar en celda separada)\n    from google.colab import drive\n    drive.mount('/content/drive')\n    \n    # PASO 2: Cambiar al directorio de trabajo\n    %cd /content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones\n    \n    # PASO 3: Importar configuracion\n    from analisis_fase_2e_config import *\n    \n   

In [ ]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import os
import sys
import logging
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Backend no interactivo para Colab
import matplotlib.pyplot as plt
import seaborn as sns

# Configuracion de warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# =============================================================================
# CONFIGURACION DE RUTAS
# =============================================================================

def detectar_entorno() -> tuple:
    """
    Detecta el entorno de ejecucion y retorna la ruta base.

    Returns:
        Tupla (ruta_base, nombre_entorno)
    """
    # Google Colab con Drive montado
    if os.path.exists('/content/drive/MyDrive'):
        return Path('/content/drive/MyDrive/TFM'), 'colab'
    # Entorno Claude/desarrollo
    elif os.path.exists('/mnt/user-data/uploads'):
        return Path('/mnt/user-data/uploads'), 'claude'
    # Local - buscar estructura TFM
    else:
        cwd = Path.cwd()
        # Buscar si estamos dentro de estructura TFM
        for parent in [cwd] + list(cwd.parents):
            if (parent / '0_Imagenes').exists() or (parent / '3_Analisis').exists():
                return parent, 'local'
        return cwd, 'local'


def montar_drive_colab():
    """
    Monta Google Drive en Colab si no esta montado.
    Llamar esta funcion al inicio del notebook.
    """
    try:
        from google.colab import drive
        if not os.path.exists('/content/drive/MyDrive'):
            print("Montando Google Drive...")
            drive.mount('/content/drive')
            print("Drive montado correctamente")
        else:
            print("Drive ya esta montado")
        return True
    except ImportError:
        print("No estamos en Google Colab")
        return False
    except Exception as e:
        print(f"Error montando Drive: {e}")
        return False


def configurar_path_colab():
    """
    Configura sys.path para permitir imports en Colab.
    Llamar despues de montar Drive.
    """
    ruta_analisis = Path('/content/drive/MyDrive/TFM/3_Analisis')
    ruta_fase2e = ruta_analisis / 'fase2e_visualizaciones'

    for ruta in [ruta_analisis, ruta_fase2e]:
        if ruta.exists() and str(ruta) not in sys.path:
            sys.path.insert(0, str(ruta))
            print(f"Agregado al path: {ruta}")


# Detectar entorno
RUTA_BASE, ENTORNO = detectar_entorno()

# Informar entorno detectado
print(f"Entorno detectado: {ENTORNO}")
print(f"Ruta base: {RUTA_BASE}")

# Estructura de directorios
RUTAS = {
    # Datos de entrada
    'imagenes': RUTA_BASE / '0_Imagenes',
    'ground_truth': RUTA_BASE / '0_Imagenes_CVAT' / 'ground_truth_masks',
    'caracteristicas': RUTA_BASE / '1_Caracteristicas' / 'json',
    'modelos': RUTA_BASE / '2_Modelos',

    # Resultados de fases anteriores
    'fase1': RUTA_BASE / '3_Analisis' / 'fase1_integracion',
    'fase2a': RUTA_BASE / '3_Analisis' / 'fase2a_metricas',
    'fase2b': RUTA_BASE / '3_Analisis' / 'fase2b_correlaciones',
    'fase2c': RUTA_BASE / '3_Analisis' / 'fase2c_pca_clustering',
    'fase2d': RUTA_BASE / '3_Analisis' / 'fase2d_configuraciones',

    # Salida Fase 2E (PERSISTENTE EN DRIVE)
    'fase2e': RUTA_BASE / '3_Analisis' / 'fase2e_visualizaciones',
    'output_bloque1': RUTA_BASE / '3_Analisis' / 'fase2e_visualizaciones' / 'bloque1_estadisticas',
    'output_bloque2': RUTA_BASE / '3_Analisis' / 'fase2e_visualizaciones' / 'bloque2_dashboards',
    'output_bloque3': RUTA_BASE / '3_Analisis' / 'fase2e_visualizaciones' / 'bloque3_mascaras',
    'output_bloque4': RUTA_BASE / '3_Analisis' / 'fase2e_visualizaciones' / 'bloque4_catalogo',
}

# Archivos de datos principales
ARCHIVOS = {
    # Fase 2B - Correlaciones
    'metricas_fusionadas': RUTAS['fase2b'] / 'metricas_fusionadas.csv',
    'correlaciones_globales': RUTAS['fase2b'] / 'correlaciones_globales.csv',
    'correlaciones_por_modelo': RUTAS['fase2b'] / 'correlaciones_por_modelo.csv',

    # Fase 2C - PCA y Clustering (subdirectorios)
    'clusters_fotografias': RUTAS['fase2c'] / 'clustering' / 'clusters_fotografias.csv',
    'pca_componentes': RUTAS['fase2c'] / 'pca_global' / 'pca_scores.csv',
    'matriz_correlaciones': RUTAS['fase2c'] / 'correlaciones' / 'matriz_correlaciones.csv',

    # Fase 2D - Configuraciones
    'ranking_global': RUTAS['fase2d'] / 'ranking_global_top30.csv',
    'sensibilidad_umbrales': RUTAS['fase2d'] / 'sensibilidad_umbrales.csv',
    'anova_por_modelo': RUTAS['fase2d'] / 'anova_por_modelo.csv',

    # Fase 1 - Indice maestro
    'indice_maestro': RUTAS['fase1'] / 'indice_maestro.json',
}


Entorno detectado: local
Ruta base: /content


In [ ]:
# =============================================================================
# CONFIGURACION DE ESTILOS
# =============================================================================

# Parametros de estilo matplotlib
STYLE_CONFIG = {
    'figure.dpi': 300,
    'figure.facecolor': 'white',
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif', 'serif'],
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'axes.titleweight': 'bold',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
    'legend.frameon': True,
    'legend.framealpha': 0.9,
    'figure.titlesize': 14,
    'figure.titleweight': 'bold',
}

# Tamanos de figura estandar (pulgadas)
FIGSIZE = {
    'pequena': (8, 5),
    'mediana': (10, 6),
    'grande': (12, 8),
    'extra_grande': (14, 10),
    'dashboard': (16, 12),
    'cuadrada': (8, 8),
    'panoramica': (14, 5),
}

# Configuracion de exportacion
EXPORT_CONFIG = {
    'dpi': 300,
    'formatos': ['png', 'pdf'],
    'bbox_inches': 'tight',
    'pad_inches': 0.1,
    'facecolor': 'white',
    'edgecolor': 'none',
}

In [ ]:
# =============================================================================
# PALETAS DE COLORES
# =============================================================================

# Paleta principal por modelo (colorblind-friendly - Wong palette)
PALETTE_MODELS = {
    'yolov8': '#0077BB',       # Azul
    'oneformer': '#EE7733',    # Naranja
    'sam2_prompts': '#009988', # Verde azulado
    'sam2_auto': '#33BBEE',    # Cyan
    'mask2former': '#CC3311',  # Rojo
    'bodypix': '#AA3377',      # Magenta
}

# Orden de modelos para visualizaciones consistentes
ORDEN_MODELOS = ['yolov8', 'oneformer', 'sam2_prompts', 'sam2_auto', 'mask2former', 'bodypix']

# Nombres legibles para etiquetas
NOMBRES_MODELOS = {
    'yolov8': 'YOLOv8-seg',
    'oneformer': 'OneFormer',
    'sam2_prompts': 'SAM2 (Prompts)',
    'sam2_auto': 'SAM2 (Auto)',
    'mask2former': 'Mask2Former',
    'bodypix': 'BodyPix',
}

# Paleta para clusters de dificultad
PALETTE_CLUSTERS = {
    0: '#2ECC71',  # Verde - Facil
    1: '#F39C12',  # Amarillo - Medio
    2: '#E74C3C',  # Rojo - Dificil
    'facil': '#2ECC71',
    'medio': '#F39C12',
    'dificil': '#E74C3C',
}

# Paleta para overlay de mascaras (TP/FP/FN)
PALETTE_OVERLAY = {
    'tp': '#2ECC71',  # Verde - True Positive
    'fp': '#3498DB',  # Azul - False Positive
    'fn': '#E74C3C',  # Rojo - False Negative
    'gt_contorno': '#000000',    # Negro - Contorno GT
    'pred_contorno': '#FFFFFF',  # Blanco - Contorno prediccion
}

# Paleta para arquitecturas
PALETTE_ARQUITECTURAS = {
    'CNN': '#3498DB',
    'Transformer': '#E74C3C',
    'Foundation': '#2ECC71',
    'Web': '#9B59B6',
}

In [ ]:
# =============================================================================
# CONFIGURACION DE LOGGING
# =============================================================================

def configurar_logging(nombre_modulo: str = 'Fase2E') -> logging.Logger:
    """
    Configura sistema de logging sin duplicacion.

    Args:
        nombre_modulo: Nombre del modulo para el logger

    Returns:
        Logger configurado
    """
    logger = logging.getLogger(nombre_modulo)

    # Evitar duplicacion de handlers
    if logger.handlers:
        return logger

    logger.setLevel(logging.INFO)

    # Handler para consola
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)

    # Formato
    formato = logging.Formatter(
        '[%(asctime)s] %(levelname)s - %(message)s',
        datefmt='%H:%M:%S'
    )
    handler.setFormatter(formato)
    logger.addHandler(handler)

    return logger

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def aplicar_estilo():
    """Aplica configuracion de estilo global a matplotlib."""
    plt.rcParams.update(STYLE_CONFIG)
    sns.set_style("whitegrid")
    sns.set_palette("colorblind")


def crear_directorios_salida():
    """Crea los directorios de salida si no existen."""
    for nombre, ruta in RUTAS.items():
        if nombre.startswith('output_') or nombre == 'fase2e':
            ruta.mkdir(parents=True, exist_ok=True)


def guardar_figura(
    fig: plt.Figure,
    nombre: str,
    directorio: Path,
    formatos: List[str] = None,
    logger: logging.Logger = None
) -> List[Path]:
    """
    Guarda figura en multiples formatos.

    Args:
        fig: Figura matplotlib
        nombre: Nombre base del archivo (sin extension)
        directorio: Directorio de salida
        formatos: Lista de formatos ['png', 'pdf']. Por defecto ambos.
        logger: Logger para mensajes

    Returns:
        Lista de rutas de archivos guardados
    """
    if formatos is None:
        formatos = EXPORT_CONFIG['formatos']

    directorio = Path(directorio)
    directorio.mkdir(parents=True, exist_ok=True)

    archivos_guardados = []

    for fmt in formatos:
        ruta = directorio / f"{nombre}.{fmt}"
        fig.savefig(
            ruta,
            dpi=EXPORT_CONFIG['dpi'],
            bbox_inches=EXPORT_CONFIG['bbox_inches'],
            pad_inches=EXPORT_CONFIG['pad_inches'],
            facecolor=EXPORT_CONFIG['facecolor'],
            edgecolor=EXPORT_CONFIG['edgecolor'],
            format=fmt
        )
        archivos_guardados.append(ruta)

        if logger:
            logger.info(f"Guardado: {ruta.name}")

    return archivos_guardados


def cargar_datos_metricas(logger: logging.Logger = None) -> pd.DataFrame:
    """
    Carga el archivo principal de metricas fusionadas.

    Returns:
        DataFrame con metricas fusionadas
    """
    ruta = ARCHIVOS['metricas_fusionadas']

    if not ruta.exists():
        raise FileNotFoundError(f"No se encuentra: {ruta}")

    df = pd.read_csv(ruta)

    if logger:
        logger.info(f"Cargadas metricas: {len(df)} registros")

    return df


def cargar_correlaciones(logger: logging.Logger = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Carga archivos de correlaciones.

    Returns:
        Tupla (correlaciones_globales, correlaciones_por_modelo)
    """
    df_global = pd.read_csv(ARCHIVOS['correlaciones_globales'])
    df_modelo = pd.read_csv(ARCHIVOS['correlaciones_por_modelo'])

    if logger:
        logger.info(f"Cargadas correlaciones globales: {df_global.shape}")
        logger.info(f"Cargadas correlaciones por modelo: {df_modelo.shape}")

    return df_global, df_modelo


def cargar_clusters(logger: logging.Logger = None) -> pd.DataFrame:
    """
    Carga archivo de clusters de fotografias.

    Returns:
        DataFrame con asignacion de clusters
    """
    ruta = ARCHIVOS['clusters_fotografias']

    if not ruta.exists():
        if logger:
            logger.warning(f"No se encuentra clusters: {ruta}")
        return None

    df = pd.read_csv(ruta)

    if logger:
        logger.info(f"Cargados clusters: {len(df)} fotografias")

    return df


def cargar_pca(logger: logging.Logger = None) -> pd.DataFrame:
    """
    Carga componentes PCA.

    Returns:
        DataFrame con componentes PCA
    """
    ruta = ARCHIVOS['pca_componentes']

    if not ruta.exists():
        if logger:
            logger.warning(f"No se encuentra PCA: {ruta}")
        return None

    df = pd.read_csv(ruta)

    if logger:
        logger.info(f"Cargados PCA: {df.shape}")

    return df


def obtener_color_modelo(modelo: str) -> str:
    """
    Obtiene el color asignado a un modelo.

    Args:
        modelo: Nombre del modelo

    Returns:
        Codigo de color hexadecimal
    """
    # Normalizar nombre
    modelo_norm = modelo.lower().replace('-', '_').replace(' ', '_')

    # Buscar coincidencia
    for key, color in PALETTE_MODELS.items():
        if key in modelo_norm or modelo_norm in key:
            return color

    # Color por defecto
    return '#7F8C8D'


def obtener_nombre_modelo(modelo: str) -> str:
    """
    Obtiene el nombre legible de un modelo.

    Args:
        modelo: Codigo del modelo

    Returns:
        Nombre legible para etiquetas
    """
    modelo_norm = modelo.lower().replace('-', '_').replace(' ', '_')

    for key, nombre in NOMBRES_MODELOS.items():
        if key in modelo_norm or modelo_norm in key:
            return nombre

    return modelo


def extraer_modelo_de_config(config_codigo: str) -> str:
    """
    Extrae el nombre del modelo desde el codigo de configuracion.

    Args:
        config_codigo: Codigo de configuracion (ej: 'yolov8_xlarge_quality')

    Returns:
        Nombre del modelo normalizado
    """
    config_lower = config_codigo.lower()

    if 'yolo' in config_lower:
        return 'yolov8'
    elif 'oneformer' in config_lower or config_lower.startswith('of_'):
        return 'oneformer'
    elif 'sam2' in config_lower or config_lower.startswith('s2'):
        if 'prompt' in config_lower or config_lower.startswith('s2p'):
            return 'sam2_prompts'
        else:
            return 'sam2_auto'
    elif 'mask2former' in config_lower or config_lower.startswith('m2f'):
        return 'mask2former'
    elif 'bodypix' in config_lower or config_lower.startswith('bp'):
        return 'bodypix'
    else:
        return 'desconocido'


def agregar_columna_modelo(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega columna 'modelo_norm' al DataFrame basado en config_codigo.

    Args:
        df: DataFrame con columna 'config_codigo'

    Returns:
        DataFrame con columna 'modelo_norm' agregada
    """
    if 'config_codigo' in df.columns:
        df['modelo_norm'] = df['config_codigo'].apply(extraer_modelo_de_config)
    elif 'modelo' in df.columns:
        df['modelo_norm'] = df['modelo'].apply(lambda x: x.lower().replace('-', '_'))

    return df


def formatear_metrica(valor: float, decimales: int = 4) -> str:
    """
    Formatea un valor de metrica para visualizacion.

    Args:
        valor: Valor numerico
        decimales: Numero de decimales

    Returns:
        String formateado
    """
    if pd.isna(valor):
        return "N/A"
    return f"{valor:.{decimales}f}"


def generar_timestamp() -> str:
    """Genera timestamp para nombres de archivo."""
    return datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# =============================================================================
# FOTOGRAFIAS SELECCIONADAS MANUALMENTE
# =============================================================================

# Fotografias seleccionadas para dashboards y analisis detallado
FOTOS_SELECCIONADAS = [
    '_DSC0023',
    '_DSC0584',
    '_DSC0119',
    '_DSC0411',
    '_DSC0962',
    '_DSC0071',
    '_DSC0139',
    '_DSC0472',
]

In [ ]:
# =============================================================================
# INICIALIZACION
# =============================================================================

def inicializar_fase2e(logger: logging.Logger = None) -> Dict[str, Any]:
    """
    Inicializa el entorno para Fase 2E.

    Returns:
        Diccionario con estado de inicializacion
    """
    if logger is None:
        logger = configurar_logging('Fase2E_Init')

    logger.info("=" * 60)
    logger.info("INICIALIZANDO FASE 2E - VISUALIZACIONES")
    logger.info("=" * 60)

    # Aplicar estilos
    aplicar_estilo()
    logger.info("Estilos matplotlib aplicados")

    # Crear directorios
    crear_directorios_salida()
    logger.info("Directorios de salida creados")

    # Verificar archivos de entrada
    archivos_ok = {}
    for nombre, ruta in ARCHIVOS.items():
        existe = ruta.exists()
        archivos_ok[nombre] = existe
        estado = "OK" if existe else "FALTA"
        logger.info(f"  {nombre}: {estado}")

    # Resumen
    n_ok = sum(archivos_ok.values())
    n_total = len(archivos_ok)
    logger.info(f"Archivos disponibles: {n_ok}/{n_total}")

    return {
        'rutas': RUTAS,
        'archivos': archivos_ok,
        'timestamp': generar_timestamp(),
    }

In [ ]:
# =============================================================================
# PUNTO DE ENTRADA PARA PRUEBAS
# =============================================================================

if __name__ == "__main__":
    # Prueba de configuracion
    logger = configurar_logging('ConfigTest')
    estado = inicializar_fase2e(logger)

    logger.info("\nRutas configuradas:")
    for nombre, ruta in RUTAS.items():
        logger.info(f"  {nombre}: {ruta}")

    logger.info("\nPaleta de modelos:")
    for modelo, color in PALETTE_MODELS.items():
        logger.info(f"  {modelo}: {color}")

    logger.info("\nConfiguracion completada correctamente")

[23:41:26] INFO - ============================================================


INFO:ConfigTest:============================================================


[23:41:26] INFO - INICIALIZANDO FASE 2E - VISUALIZACIONES


INFO:ConfigTest:INICIALIZANDO FASE 2E - VISUALIZACIONES


[23:41:26] INFO - ============================================================


...

INFO:ConfigTest:  mask2former: #CC3311


[23:41:26] INFO -   bodypix: #AA3377


INFO:ConfigTest:  bodypix: #AA3377


[23:41:26] INFO - 
Configuracion completada correctamente


INFO:ConfigTest:
Configuracion completada correctamente
